In [38]:
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from extractor.tiingo_extractor import TiingoExtractor
from tqdm import tqdm
from time import sleep
import pickle
import pandas_datareader.data as web

In [10]:
strategy = Strategy()
strategy.connect()
categories = strategy.retrieve("application_stock_categories")
models = strategy.retrieve("models")
strategy.disconnect()
models["model"] = [pickle.loads(x) for x in models["model"]]
quarter = datetime.now().month // 3 + 1
year = datetime.now().year

In [12]:

ticker = "MSFT"
ticker_category = categories[(categories["ticker"]==ticker) & (categories["year"] == year) & (categories["quarter"]==quarter)]
number_of_training_weeks = 14
start = (datetime.now() - timedelta(days=(number_of_training_weeks + 1)*7)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
model = models[models["category"]==ticker_category["prediction"].item()].tail(1)
ticker_data = TiingoExtractor.extract(ticker,start,end)
ticker_data["date"] = pd.to_datetime(ticker_data["date"])
ticker_data["year"] = [x.year for x in ticker_data["date"]]
ticker_data["quarter"] = [x.quarter for x in ticker_data["date"]]
ticker_data["week"] = [x.week for x in ticker_data["date"]]
weekly = ticker_data.groupby(["year","quarter","week"]).mean().reset_index()
for i in range(number_of_training_weeks):
    weekly[i] = weekly["adjClose"].shift(1)
weekly["y"] = weekly["adjClose"].shift(-1)
weekly.dropna(inplace=True)
weekly["ticker"] = ticker
for i in range(number_of_training_weeks):
    weekly.rename(columns={i:str(i)},inplace=True)
X = weekly[[str(x) for x in range(number_of_training_weeks)]].tail(1)
current = ticker_data.tail(1)["adjClose"].item()
prediction = model["model"].item().predict(X)[0]

In [49]:
{"ticker":ticker,"adjClose":current,"prediction":prediction,"delta":(prediction-current)/current}